In [1]:
import numpy as np
import pandas as pd
from scipy.fftpack import dct
from sklearn.decomposition import PCA

# Read the PMU data from a CSV file
data = pd.read_csv('test.csv')

# Define disturbance detection thresholds
threshold_v = 1.0  # Voltage threshold
threshold_f = 59.9  # Frequency threshold
threshold_df_dt = 0.1  # Rate of change of frequency threshold

data


,Time,Bus ODESSA 2 0 V pu,Bus ODESSA 2 0 V angle,Bus ODESSA 2 0 Frequency,Bus PRESIDIO 2 0 V pu,Bus PRESIDIO 2 0 V angle,Bus PRESIDIO 2 0 Frequency,Bus O DONNELL 1 0 V pu,Bus O DONNELL 1 0 V angle,Bus O DONNELL 1 0 Frequency,...,Bus MCCAMEY 2 0 Frequency,Bus LAMESA 0 V pu,Bus LAMESA 0 V angle,Bus LAMESA 0 Frequency,Bus GOLDSMITH 0 V pu,Bus GOLDSMITH 0 V angle,Bus GOLDSMITH 0 Frequency,Bus RALLS 2 0 V pu,Bus RALLS 2 0 V angle,Bus RALLS 2 0 Frequency
0,0.000000,0.9842,-23.0372,60.0000,1.0225,-19.3905,60.0000,1.0131,-17.5830,60.0000,...,60.0000,0.9874,-21.7774,60.0000,1.0200,-16.6213,60.0000,1.0318,-31.5607,60.0000
1,0.033333,0.9843,-23.0325,60.0002,1.0225,-19.3836,60.0002,1.0131,-17.5823,60.0000,...,60.0000,0.9875,-21.7757,60.0001,1.0201,-16.6137,60.0003,1.0318,-31.5607,60.0000
2,0.066667,0.9843,-23.0284,60.0003,1.0226,-19.3783,60.0003,1.0131,-17.5817,60.0000,...,60.0003,0.9875,-21.7746,60.0001,1.0201,-16.6109,60.0003,1.0318,-31.5602,60.0000
3,0.100000,0.9843,-23.0244,60.0003,1.0226,-19.3731,60.0003,1.0131,-17.5802,60.0001,...,60.0003,0.9875,-21.7731,60.0001,1.0201,-16.6077,60.0003,1.0318,-31.5600,60.0000
4,0.133333,0.9843,-23.0273,60.0000,1.0227,-19.3696,60.0003,1.0131,-17.5806,60.0000,...,60.0000,0.9875,-21.7734,60.0001,1.0201,-16.6081,60.0001,1.0318,-31.5608,60.0000
5,0.166667,0.9843,-23.0258,60.0001,1.0226,-19.3701,60.0001,1.0131,-17.5801,60.0001,...,59.9998,0.9875,-21.7728,60.0001,1.0201,-16.6051,60.0002,1.0318,-31.5602,60.0000
6,0.200000,0.9843,-23.0238,60.0001,1.0226,-19.3732,59.9999,1.0131,-17.5792,60.0001,...,60.0000,0.9874,-21.7724,60.0001,1.0201,-16.6060,60.0001,1.0318,-31.5609,60.0000
7,0.233333,0.9843,-23.0247,60.0000,1.0225,-19.3833,59.9995,1.0130,-17.5825,59.9999,...,59.9999,0.9874,-21.7777,59.9998,1.0201,-16.6058,60.0000,1.0318,-31.5627,59.9999


In [2]:
data.shape

(8, 274)

In [4]:
# Function to detect disturbances
def detect_disturbance(data, threshold_v, threshold_f, threshold_df_dt):
    voltage_columns = [col for col in data.columns if 'V pu' in col]
    frequency_columns = [col for col in data.columns if 'Frequency' in col]

    disturbance_indices = []
    for v_col, f_col in zip(voltage_columns, frequency_columns):
        voltage = data[v_col]
        frequency = data[f_col]
        df_dt = np.gradient(frequency)
        
        disturbance_indices.extend(
            data.index[
                (voltage >= threshold_v) |
                (frequency < threshold_f) |
                (df_dt > threshold_df_dt)
            ].tolist()
        )
    
    return disturbance_indices

# Detect disturbances
disturbance_indices = detect_disturbance(data, threshold_v, threshold_f, threshold_df_dt)

disturbance_indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,


In [5]:
# Determine the window length
def determine_window_length(disturbance_indices):
    if disturbance_indices:
        window_length = max(disturbance_indices) - min(disturbance_indices)
    else:
        window_length = 10  # Default window length for normal conditions
    return window_length

window_length = determine_window_length(disturbance_indices)

In [11]:
# Apply PCA
def apply_pca(data):
    pca = PCA(0.95)
    pca_result = pca.fit_transform(data)
    print(pca_result.shape)
    return pca_result

data           

,Time,Bus ODESSA 2 0 V pu,Bus ODESSA 2 0 V angle,Bus ODESSA 2 0 Frequency,Bus PRESIDIO 2 0 V pu,Bus PRESIDIO 2 0 V angle,Bus PRESIDIO 2 0 Frequency,Bus O DONNELL 1 0 V pu,Bus O DONNELL 1 0 V angle,Bus O DONNELL 1 0 Frequency,...,Bus MCCAMEY 2 0 Frequency,Bus LAMESA 0 V pu,Bus LAMESA 0 V angle,Bus LAMESA 0 Frequency,Bus GOLDSMITH 0 V pu,Bus GOLDSMITH 0 V angle,Bus GOLDSMITH 0 Frequency,Bus RALLS 2 0 V pu,Bus RALLS 2 0 V angle,Bus RALLS 2 0 Frequency
0,0.000000,0.9842,-23.0372,60.0000,1.0225,-19.3905,60.0000,1.0131,-17.5830,60.0000,...,60.0000,0.9874,-21.7774,60.0000,1.0200,-16.6213,60.0000,1.0318,-31.5607,60.0000
1,0.033333,0.9843,-23.0325,60.0002,1.0225,-19.3836,60.0002,1.0131,-17.5823,60.0000,...,60.0000,0.9875,-21.7757,60.0001,1.0201,-16.6137,60.0003,1.0318,-31.5607,60.0000
2,0.066667,0.9843,-23.0284,60.0003,1.0226,-19.3783,60.0003,1.0131,-17.5817,60.0000,...,60.0003,0.9875,-21.7746,60.0001,1.0201,-16.6109,60.0003,1.0318,-31.5602,60.0000
3,0.100000,0.9843,-23.0244,60.0003,1.0226,-19.3731,60.0003,1.0131,-17.5802,60.0001,...,60.0003,0.9875,-21.7731,60.0001,1.0201,-16.6077,60.0003,1.0318,-31.5600,60.0000
4,0.133333,0.9843,-23.0273,60.0000,1.0227,-19.3696,60.0003,1.0131,-17.5806,60.0000,...,60.0000,0.9875,-21.7734,60.0001,1.0201,-16.6081,60.0001,1.0318,-31.5608,60.0000
5,0.166667,0.9843,-23.0258,60.0001,1.0226,-19.3701,60.0001,1.0131,-17.5801,60.0001,...,59.9998,0.9875,-21.7728,60.0001,1.0201,-16.6051,60.0002,1.0318,-31.5602,60.0000
6,0.200000,0.9843,-23.0238,60.0001,1.0226,-19.3732,59.9999,1.0131,-17.5792,60.0001,...,60.0000,0.9874,-21.7724,60.0001,1.0201,-16.6060,60.0001,1.0318,-31.5609,60.0000
7,0.233333,0.9843,-23.0247,60.0000,1.0225,-19.3833,59.9995,1.0130,-17.5825,59.9999,...,59.9999,0.9874,-21.7777,59.9998,1.0201,-16.6058,60.0000,1.0318,-31.5627,59.9999


In [55]:
# Apply DCT compression
def apply_dct_compression(data, threshold):
    compressed_data = np.zeros_like(data)
    for i in range(data.shape[1]):
        signal = data[:, i]
        dct_coefficients = dct(signal, type=2, norm='ortho')
        significant_indices = np.where(np.abs(dct_coefficients) > threshold)[0]
        compressed_signal = np.zeros_like(signal)
        compressed_signal[significant_indices] = dct_coefficients[significant_indices]
        compressed_data[:, i] = compressed_signal
    return compressed_data


In [56]:
# Prepare data for PCA
data_values = data.drop(columns=['Time']).values

# Apply PCA
pca_result = apply_pca(data_values)

# Apply DCT compression
dct_threshold = 0.1
compressed_pmu_data = apply_dct_compression(pca_result, dct_threshold)

np.savetxt('output.csv', compressed_pmu_data, delimiter=',')
# Your compressed PMU data is now in 'compressed_pmu_data'
# You can further process or save it as needed  